In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
devices = pd.read_csv("devices.csv")
devices["certification"] = devices["certification"].replace({np.nan:"None"})
devices.to_json("devices.json", orient = "records" )

In [ ]:
nodes = pd.read_csv("nodes.csv")

In [ ]:
feature_collection_template = {"type": "FeatureCollection" }
def create_feature(row):
    coordinates = json.loads(row["coordinates"])
    return {
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": coordinates},
        "properties": {
            "id": row["id"],
            "device_id": row["device_id"],
            "name": row["name"],
            "precise": row["precise"]
        }
    }

feature_collection = nodes.apply(create_feature, axis=1).tolist()
feature_collection_template["features"] = feature_collection
feature_json = feature_collection_template
with open("nodes.geojson", "w") as f:
    json.dump(feature_json, f)

In [ ]:
nodes

In [ ]:
feature_collection_template2 = {"type": "FeatureCollection" }

def test(df):
    return df.apply(create_feature2, axis=1)

def create_feature2(row):
    next_row_index = row["id"] + 1
    
    if next_row_index < len(nodes) + 1:
        next_row = nodes.iloc[next_row_index - 1]
        next_coordinates = json.loads(next_row["coordinates"])
        coordinates = json.loads(row["coordinates"])
        cross = 0 if (coordinates[0] * next_coordinates[0] > 0) else 1
        
        return { "type": "Feature",
          "geometry": {
            "type": "LineString",
            "coordinates": [coordinates, next_coordinates]
            },
          "properties": {
            "device_id": row["device_id"],
            "source_id": int(row["id"]),
            "target_id": int(next_row["id"]),
            "cross": cross
            }
          }
    return None

In [ ]:
edges = nodes.groupby("device_id", as_index=False).apply(test).iloc[0, 1:-1].tolist()

In [ ]:
feature_collection_template = {"type": "FeatureCollection" }
feature_collection_template["features"] = edges
feature_json = feature_collection_template
with open("edges.geojson", "w") as f:
    json.dump(feature_json, f)